# Naive bayes code - scratch

In [554]:
import pandas as pd
import numpy as np


In [556]:
file_path = r"C:\Users\ChanakyaY\Downloads\golf.csv"
golf = pd.read_csv(file_path)

In [558]:
golf

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [559]:
def prior_prob(golf,label): # label is whether yes or no
    total = golf.shape[0]
    class_examp= (golf["Play"] == label).sum()
    return class_examp / total

In [560]:
prior_prob(golf, 'yes') , prior_prob(golf, 'no')

(0.6428571428571429, 0.35714285714285715)

In [561]:
PRIOR = {
    'yes' : prior_prob(golf , 'yes'),
    'no' : prior_prob(golf , 'no')
}
print(PRIOR)

{'yes': 0.6428571428571429, 'no': 0.35714285714285715}


In [562]:
def cond_prob(golf, feature, feature_value,label):
    filtered_data = golf[golf['Play'] == label]
    numerator = np.sum(filtered_data[feature] == feature_value)
    denominator = filtered_data.shape[0]
    return numerator/denominator
    

In [563]:
cond_prob(golf, 'Windy',False,'no')

0.4

In [565]:
features = list(golf.columns)[:-1] # except last
COND_PROB = {}
for label in golf['Play'].unique():
    COND_PROB[label] = {}
    for feature in features :
        COND_PROB[label][feature] = {}
        feature_values = golf[feature].unique()
        for fea_value in feature_values :
            prob = round(cond_prob(golf,feature,fea_value,label),2)
            COND_PROB[label][feature][fea_value] = prob
            print(label,feature,fea_value,prob)
    print()

no Outlook sunny 0.6
no Outlook overcast 0.0
no Outlook rainy 0.4
no Temperature hot 0.4
no Temperature mild 0.4
no Temperature cool 0.2
no Humidity high 0.8
no Humidity normal 0.2
no Windy False 0.4
no Windy True 0.6

yes Outlook sunny 0.22
yes Outlook overcast 0.44
yes Outlook rainy 0.33
yes Temperature hot 0.22
yes Temperature mild 0.44
yes Temperature cool 0.33
yes Humidity high 0.33
yes Humidity normal 0.67
yes Windy False 0.67
yes Windy True 0.33



In [566]:
COND_PROB

{'no': {'Outlook': {'sunny': 0.6, 'overcast': 0.0, 'rainy': 0.4},
  'Temperature': {'hot': 0.4, 'mild': 0.4, 'cool': 0.2},
  'Humidity': {'high': 0.8, 'normal': 0.2},
  'Windy': {False: 0.4, True: 0.6}},
 'yes': {'Outlook': {'sunny': 0.22, 'overcast': 0.44, 'rainy': 0.33},
  'Temperature': {'hot': 0.22, 'mild': 0.44, 'cool': 0.33},
  'Humidity': {'high': 0.33, 'normal': 0.67},
  'Windy': {False: 0.67, True: 0.33}}}

In [569]:
# PREDICTION

In [570]:
X_test = ['sunny','hot','normal',False]


In [571]:
for label in golf['Play'].unique():
    prior =PRIOR[label]
    liklihood = 1.0
    for i in range(len(features)):
        feature = features[i]
        fea_value =X_test[i]
        liklihood *= COND_PROB[label][feature][fea_value]
    posterior = liklihood * prior
    print(label,posterior)

no 0.006857142857142858
yes 0.013967202857142858


In [572]:
# as we didnt divide by p(b), total probability
#p(no) is
0.006/(0.006+0.013)

0.31578947368421056

In [573]:
0.013/(0.006+0.013) #p(yes)

0.6842105263157895

## NAIVES BAYES USING SKLEARN

In [578]:
# sklearn cannot learn with test data, so we convert it into number
from sklearn.preprocessing import LabelEncoder

In [580]:
le1 = LabelEncoder()
golf['Outlook']=le1.fit_transform(golf['Outlook'])

In [584]:
golf

,Outlook,Temperature,Humidity,Windy,Play
0,2,hot,high,False,no
1,2,hot,high,True,no
2,0,hot,high,False,yes
3,1,mild,high,False,yes
4,1,cool,normal,False,yes
5,1,cool,normal,True,no
6,0,cool,normal,True,yes
7,2,mild,high,False,no
8,2,cool,normal,False,yes
9,1,mild,normal,False,yes


In [585]:
le2 = LabelEncoder()
golf['Temperature']=le2.fit_transform(golf['Temperature'])

In [586]:
le3 = LabelEncoder()
golf['Humidity']=le3.fit_transform(golf['Humidity'])

In [587]:
le4 = LabelEncoder()
golf['Windy']=le4.fit_transform(golf['Windy'])

In [591]:
le5 = LabelEncoder()
golf['Play']=le5.fit_transform(golf['Play'])

In [592]:
golf # all features are converted into numeric , now we can apply naive bayes

,Outlook,Temperature,Humidity,Windy,Play
0,2,1,0,0,0
1,2,1,0,1,0
2,0,1,0,0,1
3,1,2,0,0,1
4,1,0,1,0,1
5,1,0,1,1,0
6,0,0,1,1,1
7,2,2,0,0,0
8,2,0,1,0,1
9,1,2,1,0,1


In [593]:
X = golf.iloc[:,:-1] #all rows except last col
y = golf.iloc[:,-1] # all rows only last col

In [594]:
from sklearn.naive_bayes import CategoricalNB

In [595]:
model = CategoricalNB()
print(y)

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Play, dtype: int32


In [596]:
model.fit(X,y)

CategoricalNB()

In [597]:
X_test = ['sunny','hot','normal',False]

In [600]:
X_test = np.array([[2,1,1,0]])

In [601]:
model.predict(X_test)

C:\Users\ChanakyaY\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


array([1])

In [627]:
model.predict_proba(X_test)

C:\Users\ChanakyaY\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


array([[0.33508723, 0.66491277]])

In [632]:
## ANTHE NA ANTHE NA ANTHE 